In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Acute_Myeloid_Leukemia"
cohort = "GSE161532"

# Input paths
in_trait_dir = "../../input/GEO/Acute_Myeloid_Leukemia"
in_cohort_dir = "../../input/GEO/Acute_Myeloid_Leukemia/GSE161532"

# Output paths
out_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/GSE161532.csv"
out_gene_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE161532.csv"
out_clinical_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE161532.csv"
json_path = "../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling of Acute Myeloid Leukemia"
!Series_summary	"The transcriptional profile of acute myeloid leukemia (AML) cells changes according to the disease molecular and genomic properties and to the microenvironmental features. Moreover, it shapes the interaction with the tissue and immune microenvironment. We analyzed the gene expression profile of 61 AML cases (Affymetrix Human Transcriptome Array 2.0, Thermo Fisher Scientific) in order to identify investigate the potential involvement of adrenomedullin in AML and the alterations having a putative causal and/or tolerogenic role towards aneuploidy."
!Series_summary	"The gene expression profile of 61 AML cases was determined using Affymetrix Human Transcriptome Array 2.0, in order to identify alterations with a putative causal and/or tolerogenic role towards aneuploidy."
!Series_overall_design	"Bone marow cells from AML patients (more than or equal to 80% blast cells) were used for R

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Based on Series description, this dataset contains gene expression data from microarrays

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 4  # 'disease state' can be used to determine AML status
age_row = 1    # Age information is available
gender_row = 2 # Gender information is available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert the AML disease state to binary: 1 for any type of AML, 0 otherwise."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # All entries contain "AML" as per the sample characteristics dictionary
    # So this is essentially a constant feature, but we'll keep it for completeness
    return 1 if 'AML' in value else 0

def convert_age(value):
    """Convert age to a numeric value."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to numeric, handling 'na' values
    if value.lower() == 'na':
        return None
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available (it is if trait_row is not None)
is_trait_available = trait_row is not None

# Conduct initial filtering on the usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# We'll skip clinical feature extraction for now since we don't have the properly formatted clinical data
# The function needs the original clinical data in the correct format, which isn't available in this task
print("Clinical feature extraction skipped due to missing proper clinical data format.")
print(f"Trait row: {trait_row}, Age row: {age_row}, Gender row: {gender_row}")
print(f"Trait availability: {is_trait_available}, Gene availability: {is_gene_available}")


Clinical feature extraction skipped due to missing proper clinical data format.
Trait row: 4, Age row: 1, Gender row: 2
Trait availability: True, Gene availability: True


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These are Affymetrix probe set IDs from a microarray platform, not standard human gene symbols.
# They need to be mapped to gene symbols for meaningful biological interpretation.
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examining the gene identifiers in gene expression data and gene annotation data
# From the previous steps, we can observe that gene identifiers in gene_data have the format like "2824546_st"
# In the gene annotation, we need to find which column matches this format

# Check column names in gene_annotation to identify the ID column
print("Column names in gene_annotation:")
print(gene_annotation.columns.tolist())

# Looking at gene identifiers in both datasets
print("\nFirst few gene identifiers in gene_data:")
print(gene_data.index[:5])

# Look at a sample of the probeset_id column to see if it matches our gene expression IDs
print("\nFirst few probeset_ids in gene_annotation:")
print(gene_annotation['probeset_id'].head().tolist())

# Compare with other potential identifier columns
print("\nFirst few IDs in gene_annotation:")
print(gene_annotation['ID'].head().tolist())

# After examining the data, we need to use ID from gene_annotation for mapping
# The 'gene_assignment' column contains the gene symbols but needs parsing

# Let's create a more consistent mapping by extracting gene symbols from gene_assignment
# 2. Get gene mapping dataframe
# The ID column in gene_annotation matches the probe IDs in gene_data
# For gene symbols, we need to extract from gene_assignment
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# Print the mapping to verify its structure
print("\nGene mapping dataframe (first few rows):")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the first few rows of the resulting gene expression dataframe
print("\nGene expression dataframe after mapping (first few rows):")
print(gene_data.head())


Column names in gene_annotation:
['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']

First few gene identifiers in gene_data:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st'], dtype='object', name='ID')

First few probeset_ids in gene_annotation:
['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1']

First few IDs in gene_annotation:
['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1']



Gene mapping dataframe (first few rows):
                ID                                               Gene
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...
2  TC01000003.hg.1  NM_001005484 // OR4F5 // olfactory receptor, f...
3  TC01000004.hg.1  OTTHUMT00000007169 // OTTHUMG00000002525 // NU...
4  TC01000005.hg.1  NR_028322 // LOC100132287 // uncharacterized L...



Gene expression dataframe after mapping (first few rows):
         GSM4909492  GSM4909493  GSM4909494  GSM4909495  GSM4909496  \
Gene                                                                  
A-        19.717014   19.988207   20.262083   21.016678   20.559258   
A-2        0.941636    0.916887    0.891820    0.915832    0.886529   
A-52       5.242986    5.407402    5.369311    5.141263    5.422811   
A-575C2    2.368492    2.362056    2.408504    2.332893    2.431963   
A-E        2.211264    1.684236    2.269916    1.997309    2.201032   

         GSM4909497  GSM4909498  GSM4909499  GSM4909500  GSM4909501  ...  \
Gene                                                                 ...   
A-        20.374758   19.327033   19.760321   21.080433   19.348171  ...   
A-2        0.922684    0.956801    0.998681    0.920608    0.912332  ...   
A-52       5.427153    5.296226    5.240743    5.369918    5.294553  ...   
A-575C2    2.223798    2.063372    2.656254    2.334759    2.13

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the determined rows and conversion functions from Step 2
clinical_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait,
    trait_row=4,
    convert_trait=convert_trait,
    age_row=1,
    convert_age=convert_age,
    gender_row=2,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(clinical_df))

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Verify that the trait column has at least two unique values
unique_trait_values = linked_data[trait].unique()
print(f"Unique values in trait column: {unique_trait_values}")

# 5. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
note = "Dataset contains AML (Acute Myeloid Leukemia) samples with different subtypes."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Processed dataset saved to {out_data_file}")
else:
    print("Dataset not usable due to bias in trait distribution. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE161532.csv
Clinical data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE161532.csv
Clinical data shape: (3, 61)
Clinical data preview:
{'GSM4909492': [1.0, 54.0, 0.0], 'GSM4909493': [1.0, 66.0, 1.0], 'GSM4909494': [1.0, 65.0, 1.0], 'GSM4909495': [1.0, 38.0, 0.0], 'GSM4909496': [1.0, nan, 0.0], 'GSM4909497': [1.0, 51.0, 0.0], 'GSM4909498': [1.0, 82.0, 1.0], 'GSM4909499': [1.0, 70.0, 1.0], 'GSM4909500': [1.0, nan, 1.0], 'GSM4909501': [1.0, 69.0, 1.0], 'GSM4909502': [1.0, 72.0, 0.0], 'GSM4909503': [1.0, 59.0, 0.0], 'GSM4909504': [1.0, 47.0, 0.0], 'GSM4909505': [1.0, 67.0, 1.0], 'GSM4909506': [1.0, 63.0, 0.0], 'GSM4909507': [1.0, 42.0, 0.0], 'GSM4909508': [1.0, 71.0, 1.0], 'GSM4909509': [1.0, 64.0, 1.0], 'GSM4909510': [1.0, 57.0, 0.0], 'GSM4909511': [1.0, 70.0, 0.0], 'GSM4909512': [1.0, 62.0, 1.0], 'GSM4909513': [1.0, 66.0, 0.0], 'GSM4909514': [1.0, 60.0, 0.0], 'GSM4909515': 

Data shape after handling missing values: (61, 24021)
Unique values in trait column: [1.]
Quartiles for 'Acute_Myeloid_Leukemia':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Acute_Myeloid_Leukemia' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 63.0
  75%: 68.0
Min: 31.0
Max: 82.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 30 occurrences. This represents 49.18% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset not usable due to bias in trait distribution. Data not saved.
